In [65]:
import tensorflow as tf
import csv
import random
from collections import Counter

In [66]:
groups = ['Pink','Purple','Red','Orange','Yellow','Green','Cyan','Blue','Brown','White',"Grey"]

data=[]

with open('colors.csv','r') as f:
    reader = csv.reader(f,delimiter=',')
    next(reader) # Skip headings
    for row in reader:
        # print(row)
        data.append([int(row[2].strip()),int(row[3].strip()),int(row[4].strip()),groups.index(row[-1].strip())])

# print(data)
random.shuffle(data)
data_tensor = tf.constant(data)
# print(data_tensor)


Groups reference taken from [https://www.w3schools.com/colors/colors_groups.asp](https://www.w3schools.com/colors/colors_groups.asp)

In [67]:
def from_hex(h):
    h=str(h)
    start_index = 0
    if h[0] == '#':
        start_index = 1
    return [
        int(h[start_index:start_index+2],16),
        int(h[start_index+2:start_index+4],16),
        int(h[start_index+4:start_index+6],16)
    ]

In [69]:
def KNN(dataset, color ,k=5, sample_size=None, test_size=None):
    if type(color) == str:
        color = from_hex(color)
    if type(color) == list:
        color = tf.constant(color)
    # print(color)

    features = tf.gather(dataset,[0,1,2],axis=1)
    labels = tf.gather(dataset,[3],axis=1)

    diff = (features - color)
    # print(diff)
    diff_squared = diff**2
    # print(diff_squared)
    sums = tf.reduce_sum(diff_squared,axis=1)
    # print(sums.shape)
    sums = tf.reshape(sums,[-1,1])
    # print(sums.shape)
    # print(sums)
    diff_squared_sum = tf.concat([sums,labels],axis=1)
    # print(diff_squared_sum)
    diff_squared_sum_sorted = sorted(diff_squared_sum.numpy().tolist(),key=lambda r:r[0])
    # print(diff_squared_sum_sorted)

    top_k = [_[1] for _ in diff_squared_sum_sorted[:k]]
    # print(top_k)
    
    most_frequent = Counter(top_k).most_common(1) # get 1 of most frequent

    return most_frequent[0][0]


print(groups[KNN(data_tensor,tf.constant([246,234,254]),5)])
print(groups[KNN(data_tensor,[1,1,1],5)])
print(groups[KNN(data_tensor,from_hex('#40E0D0'))])

White
Grey
Cyan
